## 1. Import packages

In [ ]:
import pandas as pd
import glob
import os
import time

In [2]:
%cd /data/p_dsi/teams2023/team7/remove-duplicates/

/gpfs52/data/p_dsi/teams2023/team7/remove-duplicates


## 2. Subset tire sales

### 2.1 Subset by CATEGORY_CODE

In [4]:
product = pd.read_csv('/data/p_dsi/teams2023/bridgestone_data/data/product.csv', sep = '|')

In [5]:
tire_sales = product[(product["CATEGORY_CODE"] == 44.0) | (product["CATEGORY_CODE"] == 26.0)]

### 2.2 Get all the ariticle_ids for tire sales

In [6]:
ARTICLE_ID_list = tire_sales["ARTICLE_ID"].unique().tolist()

In [8]:
csv_files = ['sales_20171130.csv','sales_20171231.csv']

In [9]:
master_tire_sales_2017 = pd.DataFrame()
for file in csv_files:
    sales_2017 = pd.read_csv(file)
    sales_small = sales_2017[["ARTICLE_ID", "UNITS"]]
    tire_sales_2017 = sales_small[sales_small['ARTICLE_ID'].isin(ARTICLE_ID_list)]
    df = tire_sales_2017.groupby(by=["ARTICLE_ID"]).sum()
    master_tire_sales_2017 = pd.concat([master_tire_sales_2017, df])

In [10]:
master_tire_sales_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7087 entries, 9 to 7099819
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   UNITS   7087 non-null   float64
dtypes: float64(1)
memory usage: 110.7 KB


In [11]:
master_tire_sales_2017 = master_tire_sales_2017.groupby(by=["ARTICLE_ID"]).sum()
master_tire_sales_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4145 entries, 9 to 7099982
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   UNITS   4145 non-null   float64
dtypes: float64(1)
memory usage: 64.8 KB


In [12]:
csv_files = glob.glob('*sales_2018*.csv')

In [17]:
master_tire_sales_2018 = pd.DataFrame()
for file in csv_files:
    sales_2018 = pd.read_csv(file)
    print(file)
    sales_small = sales_2018[["ARTICLE_ID", "UNITS"]]
    tire_sales_2018 = sales_small[sales_small['ARTICLE_ID'].isin(ARTICLE_ID_list)]
    df = tire_sales_2018.groupby(by=["ARTICLE_ID"]).sum()
    master_tire_sales_2018 = pd.concat([master_tire_sales_2018, df])

sales_20180731.csv
sales_20180831.csv
sales_20180531.csv
sales_20180430.csv
sales_20180630.csv
sales_20180228.csv
sales_20180131.csv
sales_20181031.csv
sales_20180930.csv
sales_20180331.csv


In [18]:
master_tire_sales_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39146 entries, 9 to 7099847
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   UNITS   39146 non-null  float64
dtypes: float64(1)
memory usage: 611.7 KB


### 2.3 Get the article_ids whose sales are greater than 5000 from 2017 to 2018

In [19]:
unit_tire_sales_year = pd.concat([master_tire_sales_2017, master_tire_sales_2018])
unit_tire_sales_year = unit_tire_sales_year.groupby(by=["ARTICLE_ID"]).sum()

unit_tire_sales_year = unit_tire_sales_year.loc[unit_tire_sales_year['UNITS'] > 5000]
unit_tire_sales_year.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 33 to 7099817
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   UNITS   420 non-null    float64
dtypes: float64(1)
memory usage: 6.6 KB


### 2.4 Save the aritcle_id list

In [20]:
#save the final result in csv file
unit_tire_sales_year.to_csv("/data/p_dsi/teams2023/team7/no_duplicate_tire_sales_article_id_and_total_units_last_12_months.csv")

In [4]:
article_id_units = pd.read_csv("/data/p_dsi/teams2023/team7/no_duplicate_tire_sales_article_id_and_total_units_last_12_months.csv", sep = ',')
article_ids = article_id_units["ARTICLE_ID"].tolist()

In [5]:
len(article_ids)

420

## 3. Save the final_sales CSVs for tire sales.

In [6]:
csv_files = []

for file in os.listdir("/data/p_dsi/teams2023/team7/remove-duplicates/"):
    if file.startswith('sales_201'):
        csv_files.append(file)

print(csv_files)
print(len(csv_files))

['sales_20180731.csv', 'sales_20150731.csv', 'sales_20170831.csv', 'sales_20170731.csv', 'sales_20150531.csv', 'sales_20160131.csv', 'sales_20151130.csv', 'sales_20170531.csv', 'sales_20160331.csv', 'sales_20170228.csv', 'sales_20160930.csv', 'sales_20170131.csv', 'sales_20180831.csv', 'sales_20160430.csv', 'sales_20170331.csv', 'sales_20171031.csv', 'sales_20180531.csv', 'sales_20160831.csv', 'sales_20170630.csv', 'sales_20161031.csv', 'sales_20160630.csv', 'sales_20170930.csv', 'sales_20180430.csv', 'sales_20161231.csv', 'sales_20171231.csv', 'sales_20180630.csv', 'sales_20180228.csv', 'sales_20180131.csv', 'sales_20160731.csv', 'sales_20170430.csv', 'sales_20150430.csv', 'sales_20181031.csv', 'sales_20161130.csv', 'sales_20180930.csv', 'sales_20150930.csv', 'sales_20150831.csv', 'sales_20160531.csv', 'sales_20151031.csv', 'sales_20180331.csv', 'sales_20160229.csv', 'sales_20151231.csv', 'sales_20150630.csv', 'sales_20171130.csv']
43


In [7]:
start_time = time.time()
no_duplicate_final_sales_dict = dict()

for file_name in csv_files:
    in_path = "/data/p_dsi/teams2023/team7/remove-duplicates/" + file_name
    sales = pd.read_csv(in_path)
    no_duplicate_final_sales = sales[sales['ARTICLE_ID'].isin(article_ids)]
    no_duplicate_final_sales_dict[file_name] = no_duplicate_final_sales
    out_path = "/data/p_dsi/teams2023/team7/no_duplicate_final_" + file_name
    print(out_path)
    no_duplicate_final_sales.to_csv(out_path)

end_time = time.time()
time_passed = end_time - start_time
print("time_passed:", time_passed)

/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20180731.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20150731.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20170831.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20170731.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20150531.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20160131.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20151130.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20170531.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20160331.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20170228.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20160930.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20170131.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20180831.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20160430.csv
/data/p_dsi/teams2023/team7/no_duplicate_final_sales_20170331.csv
/data/p_ds

In [8]:
years = [2015, 2016, 2017, 2018]
[pd.concat([no_duplicate_final_sales_dict[key] for key in no_duplicate_final_sales_dict.keys() if key.startswith("sales_{}".format(year))]).to_csv("/data/p_dsi/teams2023/team7/no_duplicate_final_sales_{}.csv".format(year)) for year in years]

[None, None, None, None]

In [9]:
pd.concat(no_duplicate_final_sales_dict.values()).to_csv("/data/p_dsi/teams2023/team7/no_duplicate_final_sales.csv")